In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset = 'manual' # mrpc, manual
model_name = 't5-small' #t5-small, t5-base

folder_input_path = '/content/drive/My Drive/Colab Notebooks/5_Corpora/corpora/'
folder_pretrained_path = f'/content/drive/My Drive/Colab Notebooks/8_Text_Paraphrasing/pretrained/{dataset}-{model_name}-cc'
csv_file_path = f'{dataset}-triplet-corpus.csv'
folder_pretrained_path

'/content/drive/My Drive/Colab Notebooks/8_Text_Paraphrasing/pretrained/manual-t5-small-cc'

In [4]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm.auto import tqdm

In [5]:
# Load the dataset
df = pd.read_csv(folder_input_path + csv_file_path)
# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
print(train_df.shape, val_df.shape)

(190, 38) (22, 38)


In [6]:
# from transformers import TFAutoModel, AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(save_directory)
# model = TFAutoModel.from_pretrained(save_directory, from_pt=True)

In [7]:
tokenizer = T5Tokenizer.from_pretrained(folder_pretrained_path)
model = T5ForConditionalGeneration.from_pretrained(folder_pretrained_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
sentiment = 'POS'
model.eval()
for index, row in val_df.iterrows():
    input_text = f"paraphrase: [{sentiment.upper()}] {row['original']}"
    input_token = tokenizer.encode(input_text, return_tensors="pt")
    generated_ids = model.generate(input_token, max_length=50, num_beams=5, early_stopping=True)
    print("original:   ", row['original'])
    print("negative:   ", row['negative']) if sentiment == 'NEG' else print("positive:   ", row['positive'])
    print("paraphrase: ", tokenizer.decode(generated_ids[0], skip_special_tokens=True))
    print("-"*100)

original:    The hotel's location is ordinary.
positive:    The hotel is situated in a reasonably accessible area.
paraphrase:  The hotel provides a peaceful retreat from urban noise.
----------------------------------------------------------------------------------------------------
original:    The painting is characterized by vivid colors.
positive:    The painting's vibrant colors bring it to life.
paraphrase:  The painting's vibrant colors bring it to life.
----------------------------------------------------------------------------------------------------
original:    He exhibits a strong ambition.
positive:    His ambition drives him to achieve remarkable goals.
paraphrase:  He consistently accomplishes with its outstanding artistic achievement.
----------------------------------------------------------------------------------------------------
original:    The lecture progressed at a reasonable pace.
positive:    The lecture progressed at a well-measured and understandable pace